<a href="https://colab.research.google.com/github/0Uriel-Rodriguez/DogsvsCatsExp/blob/main/Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://dagshub.com/jorgevc/Gatos-y-Perros.git


fatal: destination path 'Gatos-y-Perros' already exists and is not an empty directory.


In [2]:
%cd Gatos-y-Perros

/content/Gatos-y-Perros


In [3]:
pip install dvc>=2.8.1 --quiet

In [4]:
!dvc pull -r origin

ERROR: failed to collect 'workspace' - s3 is supported, but requires 'dvc-s3' to be installed: No module named 'dvc_s3'
ERROR: unexpected error - s3 is supported, but requires 'dvc-s3' to be installed: No module named 'dvc_s3'

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [5]:
!more .dvc/config

[core]
    autostage = true
['remote "origin"']
    url = s3://dvc
    endpointurl = https://dagshub.com/jorgevc/Gatos-y-Perros.s3


In [6]:
!dvc remote add origin https://dagshub.com/jorgevc/Gatos-y-Perros.dvc -f
!more .dvc/config

[core]
    autostage = true
['remote "origin"']
    url = https://dagshub.com/jorgevc/Gatos-y-Perros.dvc


In [7]:
!dvc get-url https://dagshub.com/jorgevc/Gatos-y-Perros/raw/24eaabd8ca81ed89f780138c1e3955ccd1f3d155/data/DogsCats.zip

!
  0%|          |https://dagshub.com/jorgevc/Gatos-y0.00/? [00:00<?,        ?B/s]
  0%|          |https://dagshub.com/jorgevc/Gatos-y0.00/? [00:00<?,        ?B/s]
  0%|          |https://dagshub.com/jorgevc/Ga59.1k/? [00:00<00:00,     586kB/s]
  0%|          |https://dagshub.com/jorgevc/Gat183k/? [00:00<00:00,     968kB/s]
  0%|          |https://dagshub.com/jorgevc/Gat567k/? [00:00<00:00,    2.30MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga1.69M/? [00:00<00:00,    6.08MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga5.10M/? [00:00<00:00,    16.7MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga6.69M/? [00:00<00:00,    16.5MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga8.85M/? [00:00<00:00,    17.3MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga11.5M/? [00:00<00:00,    20.5MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga12.7M/? [00:00<00:00,    17.9MB/s]
  0%|          |https://dagshub.com/jorgevc/Ga14.9M/? [00:01<00:00,    19.4MB/s]
  0%|          |https://da

In [8]:
!ls


'=2.8.1'   data.dvc   DogsCats.zip   README.md	 redconv.py   reddensa.py


In [9]:
import zipfile
# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = 'DogsCats.zip'

# Ruta del directorio donde deseas extraer los archivos
data = 'data'

# Descomprimir el archivo ZIP
with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    zip_ref.extractall(data)

print(f'Archivos descomprimidos en: {data}')

Archivos descomprimidos en: data


In [10]:
!pip install tensorflow

In [11]:
%pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [12]:
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Activation, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [13]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
#-------------------------------------------------mejor experimento---------------------------------------------------



ih, iw = 150, 150 #tamano de la imagen
input_shape = (ih, iw, 3) #forma de la imagen: alto ancho y numero de canales

#train_dir = 'data/minitrain' #directorio de entrenamiento
#test_dir = 'data/minitest' #directorio de prueba
train_dir = 'data/train' #directorio de entrenamiento
test_dir = 'data/test' #directorio de prueba




num_class = 2 #cuantas clases
epochs = 50 #cuantas veces entrenar. En cada epoch hace una mejora en los parametros
learning_rate = 0.0002
batch_size = 5 #batch para hacer cada entrenamiento. Lee 50 'batch_size' imagenes antes de actualizar los parametros. Las carga a memoria




num_train = 20000 #numero de imagenes en train
num_test = 1000 #numero de imagenes en test


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


gentrain = ImageDataGenerator(rescale=1. / 255.,
                              rotation_range=40,  # Rango de rotación aleatoria
                              width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
                              height_shift_range=0.2,  # Desplazamiento vertical aleatorio
                              shear_range=0.2,  # Deformación de corte
                              zoom_range=0.2) #indica que reescale cada canal con valor entre 0 y 1.


train = gentrain.flow_from_directory(train_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

gentest = ImageDataGenerator(rescale=1. / 255)

test = gentest.flow_from_directory(test_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

#para cargar pesos de la red desde donde se quedó la ultima vez
#filename = "cvsd.h5"
#model.load_weights(filename)  #comentar si se comienza desde cero.
###

model = Sequential()
model.add(Conv2D(10, (3, 3), input_shape=(ih, iw,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(20, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(20, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.02))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()




# Configurar w&b
wandb.init(project='DogsvsCats', force=True)
wandb.config.learning_rate = learning_rate
wandb.config.batch_size = batch_size
wandb.config.epochs = epochs
#wandb.config.metrics = metrics
#wandb.config.loss = loss
#wandb.config.optimizer = 'rmsprop'


model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

# Iniciar el contador de tiempo
start_time = time.time()

model.fit(
                train,
                steps_per_epoch=epoch_steps,
                epochs=epochs,
                validation_data=test,
                validation_steps=test_steps,
                callbacks= [WandbCallback()])


# Detener el contador de tiempo
end_time = time.time()

# Calcular el tiempo transcurrido en segundos
elapsed_time_seconds = end_time - start_time

# Convertir segundos a minutos
elapsed_time_minutes = elapsed_time_seconds / 60

# Imprimir el tiempo transcurrido en minutos
print(f"Tiempo transcurrido: {elapsed_time_minutes} minutos")



# Guardar el modelo en un archivo .h5
model.save('m3.h5')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 10)      280       
                                                                 
 activation (Activation)     (None, 148, 148, 10)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 10)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 20)        1820      
                                                                 
 activation_1 (Activation)   (None, 47, 47, 20)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 1

wandb: Currently logged in as: uriel-rodriguezs (redes). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50


KeyboardInterrupt: ignored

In [15]:
from tensorflow.keras.models import load_model

modelu = load_model('m3.h5')  # Reemplaza 'm3.h5' con la ruta de tu modelo pre-entrenado.


In [17]:
ih, iw = 150, 150 #tamano de la imagen
input_shape = (ih, iw, 3) #forma de la imagen: alto ancho y numero de canales

#train_dir = 'data/minitrain' #directorio de entrenamiento
#test_dir = 'data/minitest' #directorio de prueba
train_dir = 'data/train' #directorio de entrenamiento
test_dir = 'data/test' #directorio de prueba




num_class = 2 #cuantas clases
epochs = 25 #cuantas veces entrenar. En cada epoch hace una mejora en los parametros
learning_rate = 0.0003
batch_size = 20 #batch para hacer cada entrenamiento. Lee 50 'batch_size' imagenes antes de actualizar los parametros. Las carga a memoria




num_train = 20000 #numero de imagenes en train
num_test = 1000 #numero de imagenes en test


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


gentrain = ImageDataGenerator(rescale=1. / 255.,
                              rotation_range=40,  # Rango de rotación aleatoria
                              width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
                              height_shift_range=0.2,  # Desplazamiento vertical aleatorio
                              shear_range=0.2,  # Deformación de corte
                              zoom_range=0.2) #indica que reescale cada canal con valor entre 0 y 1.


train = gentrain.flow_from_directory(train_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

gentest = ImageDataGenerator(rescale=1. / 255)

test = gentest.flow_from_directory(test_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

#para cargar pesos de la red desde donde se quedó la ultima vez
#filename = "cvsd.h5"
#model.load_weights(filename)  #comentar si se comienza desde cero.
###

model = Sequential()
model.add(Conv2D(10, (3, 3), input_shape=(ih, iw,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(20, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(20, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.02))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()




# Configurar w&b
wandb.init(project='DogsvsCats', force=True)
wandb.config.learning_rate = learning_rate
wandb.config.batch_size = batch_size
wandb.config.epochs = epochs
#wandb.config.metrics = metrics
#wandb.config.loss = loss
#wandb.config.optimizer = 'rmsprop'


model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

# Iniciar el contador de tiempo
start_time = time.time()

modelu.fit(
                train,
                steps_per_epoch=epoch_steps,
                epochs=epochs,
                validation_data=test,
                validation_steps=test_steps,
                callbacks= [WandbCallback()])


# Detener el contador de tiempo
end_time = time.time()

# Calcular el tiempo transcurrido en segundos
elapsed_time_seconds = end_time - start_time

# Convertir segundos a minutos
elapsed_time_minutes = elapsed_time_seconds / 60

# Imprimir el tiempo transcurrido en minutos
print(f"Tiempo transcurrido: {elapsed_time_minutes} minutos")
# Guardar el modelo en un archivo .h5
model.save('m4.h5')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 10)      280       
                                                                 
 activation_12 (Activation)  (None, 148, 148, 10)      0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 49, 49, 10)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 47, 47, 20)        1820      
                                                                 
 activation_13 (Activation)  (None, 47, 47, 20)        0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None,

accuracy,▁▄▄▅▃▆▃▅▁▇▆▆█
epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,▇▇▇▆▇▅▅▄█▄▅▆▁
val_accuracy,▆▃▅▁▇▃▆▅▁█▅▄▇
val_loss,▃▇▃█▂▆▂▂▆▁▄▅▂
accuracy,0.8536
best_epoch,9
best_val_loss,0.27319
epoch,12
loss,0.3282
val_accuracy,0.885


Epoch 1/25
1000/1000 [==============================] - ETA: 0s - loss: 0.3287 - accuracy: 0.8569

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/Gatos-y-Perros/wandb/run-20231001_235044-c2qvhewk/files/model-best)... Done. 0.0s


1000/1000 [==============================] - 378s 378ms/step - loss: 0.3287 - accuracy: 0.8569 - val_loss: 0.2580 - val_accuracy: 0.8990
Epoch 2/25
1000/1000 [==============================] - 370s 369ms/step - loss: 0.3267 - accuracy: 0.8571 - val_loss: 0.2858 - val_accuracy: 0.8760
Epoch 3/25
1000/1000 [==============================] - 376s 376ms/step - loss: 0.3284 - accuracy: 0.8529 - val_loss: 0.2716 - val_accuracy: 0.8840
Epoch 4/25
1000/1000 [==============================] - 370s 369ms/step - loss: 0.3344 - accuracy: 0.8526 - val_loss: 0.2801 - val_accuracy: 0.8850
Epoch 5/25
1000/1000 [==============================] - 373s 373ms/step - loss: 0.3236 - accuracy: 0.8564 - val_loss: 0.3509 - val_accuracy: 0.8490
Epoch 6/25
1000/1000 [==============================] - 365s 365ms/step - loss: 0.3277 - accuracy: 0.8536 - val_loss: 0.3134 - val_accuracy: 0.8710
Epoch 7/25
1000/1000 [==============================] - 372s 372ms/step - loss: 0.3245 - accuracy: 0.8569 - val_loss: 0.282

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/Gatos-y-Perros/wandb/run-20231001_235044-c2qvhewk/files/model-best)... Done. 0.0s


1000/1000 [==============================] - 366s 365ms/step - loss: 0.3237 - accuracy: 0.8560 - val_loss: 0.2490 - val_accuracy: 0.9030
Epoch 12/25
1000/1000 [==============================] - 367s 367ms/step - loss: 0.3150 - accuracy: 0.8612 - val_loss: 0.2747 - val_accuracy: 0.8820
Epoch 13/25
1000/1000 [==============================] - 370s 370ms/step - loss: 0.3258 - accuracy: 0.8554 - val_loss: 0.2556 - val_accuracy: 0.8950
Epoch 14/25
1000/1000 [==============================] - 367s 367ms/step - loss: 0.3172 - accuracy: 0.8628 - val_loss: 0.2690 - val_accuracy: 0.8860
Epoch 15/25
1000/1000 [==============================] - 367s 367ms/step - loss: 0.3206 - accuracy: 0.8582 - val_loss: 0.2877 - val_accuracy: 0.8710
Epoch 16/25
1000/1000 [==============================] - 364s 364ms/step - loss: 0.3183 - accuracy: 0.8601 - val_loss: 0.2958 - val_accuracy: 0.8770
Epoch 17/25
1000/1000 [==============================] - 373s 373ms/step - loss: 0.3196 - accuracy: 0.8586 - val_loss:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
